# Experiment v1: SFT + LoRA (Unsloth)
# Base model: unsloth/Llama-3.2-3B-Instruct

In [1]:
# Install Unsloth (includes a patched, stable HF + PEFT stack)
!pip install -U unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.2/378.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/

In [2]:

import os
import json
import csv
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Global Configuration

In [10]:

MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct"
ADAPTER_DIR = "unsloth-lora-adapter"
OUTPUT_DIR = "./unsloth-output"

In [11]:
EVAL_PROMPTS = [
    "Explain LoRA in simple terms.",
    "Write a Python function to reverse a string.",
    "What is gradient accumulation and why is it used?",
    "Explain attention in transformers like I'm 12.",
    "Difference between LoRA and QLoRA.",
]

GEN_KWARGS = dict(
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9,
)

# Evaluation Utilities


In [12]:
def generate_outputs(model, tokenizer, prompts):
    FastLanguageModel.for_inference(model)
    outputs = []

    for prompt in prompts:
        full_prompt = f"""### Instruction:
{prompt}

### Response:
"""
        inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            out = model.generate(**inputs, **GEN_KWARGS)
        outputs.append(tokenizer.decode(out[0], skip_special_tokens=True))

    return outputs


def export_eval_report(prompts, before, after, prefix="eval_report"):
    rows = []
    for p, b, a in zip(prompts, before, after):
        rows.append({
            "prompt": p,
            "before": b,
            "after": a,
            "before_len": len(b.split()),
            "after_len": len(a.split()),
        })

    # JSON
    with open(f"{prefix}.json", "w") as f:
        json.dump(rows, f, indent=2)

    # CSV
    with open(f"{prefix}.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=rows[0].keys())
        writer.writeheader()
        writer.writerows(rows)


# BEFORE Evaluation

In [13]:

# -------------------------
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

before_outputs = generate_outputs(
    base_model,
    base_tokenizer,
    EVAL_PROMPTS,
)

==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Training (SFT + LoRA)

In [14]:


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)

dataset = load_dataset("tatsu-lab/alpaca", split="train[:10000]")

def format_prompt(ex):
    return f"""### Instruction:
{ex['instruction']}

### Response:
{ex['output']}"""

dataset = dataset.map(
    lambda x: {"text": format_prompt(x)},
    remove_columns=dataset.column_names,
)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    args=training_args,
)

trainer.train()

model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)

==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,587,520 of 3,217,337,344 (0.14% trained)


Step,Training Loss
10,2.018100
20,1.788200
30,1.720100
40,1.697900
50,1.722300
60,1.683000
70,1.567500
80,1.588300
90,1.674500
100,1.643500


('unsloth-lora-adapter/tokenizer_config.json',
 'unsloth-lora-adapter/special_tokens_map.json',
 'unsloth-lora-adapter/chat_template.jinja',
 'unsloth-lora-adapter/tokenizer.json')

# AFTER Evaluation

In [15]:
ft_model, ft_tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

ft_model.load_adapter(ADAPTER_DIR)

after_outputs = generate_outputs(
    ft_model,
    ft_tokenizer,
    EVAL_PROMPTS,
)


==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Export Report

In [16]:
export_eval_report(
    EVAL_PROMPTS,
    before_outputs,
    after_outputs,
    prefix="sft_v1_eval",
)

print("✅ Experiment complete. Reports saved (CSV + JSON).")

✅ Experiment complete. Reports saved (CSV + JSON).
